# **Yolo V5 Tutorial**

In this tutorial we will learn how to import and run the YOLO object detector with just a few lines of code. Let's get started!

<br>
<br>

First we will grab some data to run our object detector on. For this we will use 'wget' to obtain a video